In [ ]:
! pip install git+https://github.com/openai/whisper.git -q
! pip install pydub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
audio_file = record(60)   #will record for 60 seconds

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


In [ ]:
type(audio_file)

bytes

In [ ]:
import IPython.display as ipd

ipd.display(ipd.Audio(audio_file))

In [ ]:
with open('audio.wav', mode='bx') as f:
    f.write(audio_file)

In [ ]:
import whisper
import os
import pydub
import nltk
import shutil
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model = whisper.load_model("medium")

In [ ]:
from pydub.silence import split_on_silence

In [ ]:
options = whisper.DecodingOptions(fp16 = False)

In [ ]:
model.device

device(type='cpu')

In [ ]:
from IPython.display import Audio

In [ ]:
src = 'audio.wav'

In [ ]:
def transcribe(audio):

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # _, probs = model.detect_language(mel)
    # print(f"Detected language: {max(probs, key=probs.get)}")

    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    return result.text

In [ ]:
transcribe(src)

"I'm not Oshkosh. Okay, well what is your name? Akash. Akash. Not Oshkosh, okay? That's the company name. We don't need a company name. It's a joke. Hi, dude. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you."

In [ ]:
dst="audio.wav"
sound = pydub.AudioSegment.from_file(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='audio.wav'>

In [ ]:
def large_transcription(path):
    sound = pydub.AudioSegment.from_file(path)
    chunks = pydub.silence.split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,
        keep_silence=50,
    )
    whole_text = ""
    cf = "audio-chunks"
    if not os.path.isdir(cf):
      os.mkdir(cf)

    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(cf, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")

    for i, audio_chunk in enumerate(chunks, start=1):
        whole_text=whole_text+" "+transcribe("audio-chunks"+f"/chunk{i}.wav")
    # for i, audio_chunk in enumerate(chunks, start=1):
    #     print(transcribe("/content/openai-whisper-webapp/audio-chunks"+f"/chunk{i}.wav"), '\n')    
    return whole_text

In [ ]:
final=large_transcription(dst)
final

" Wash cars. Um, I'm not Oshkosh. Ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha Okay, well what is your name? Akash! Akash! Not Oshkosh, okay? That's the company name. We don't need a company name as a chick. Ah! Hi, Choo! Okay. So let me ask you something. The producers say... Let me know. Listen, look at him. The producers say that your dad has a bone to pick with me. What's wrong? We put that in a bottle, w

In [ ]:
def _create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [ ]:
def _score_sentences(sentences, freqTable) -> dict:
    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = len(word_tokenize(sentence))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence

    return sentenceValue

In [ ]:
def _find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]
        
    average = int(sumValues / len(sentenceValue))

    return average

In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
freq_table = _create_frequency_table(final)

sentences = sent_tokenize(final)

sentence_scores = _score_sentences(sentences, freq_table)

threshold = _find_average_score(sentence_scores)

summary = _generate_summary(sentences, sentence_scores,1.1* threshold)

print(summary)

 That's the company name. The producers say... Let me know. The producers say that your dad has a bone to pick with me. What's wrong? Stop Mr. Harvey, my dad is mad at you. For what? He's an assistant director in a pharmacy.
